In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, get_train, get_test
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [33]:
sub_type = 'AB'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,_,_ = pickle.load(f)

n_subs = np.max(params[:,0])

In [ ]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven14'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = np.max(params[:,0])
data_load = True
test = True
train = False

if test and not data_load:
    with open('real_noise/all_real_noise.p', 'rb') as f:
        real_noise_temp, _ = pickle.load(f)

clean_all = np.zeros((n_subs,8))
noisy_all = np.zeros((n_subs,8))

for sub in range(1,n_subs+1):
    print('Running ' + str(sess.sub_type) + str(sub))
    sess.sub = sub
    tf.keras.backend.clear_session()

    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)
    
    if np.sum(train_ind) and np.sum(test_ind):
        n_dof = np.max(params[train_ind,4])
    
        # Train NNs
        mlp = MLP(n_class=n_dof)
        mlp_beta = MLPbeta(n_class=n_dof)
        cnn = CNN(n_class=n_dof)

        optimizer = tf.keras.optimizers.Adam()
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

        if train:
            if data_load:
                try:
                    with open('train/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p', 'rb') as f:
                        x_train_cnn, x_train_mlp, x_train_lda, x_train_aug, y_train, y_train_lda, sess.scaler = pickle.load(f)
                except:
                    print('no training data to load')
                    data_load = False
                
                trainmlp = tf.data.Dataset.from_tensor_slices((x_train_mlp, y_train)).shuffle(x_train_mlp.shape[0],reshuffle_each_iteration=True).batch(128)
                traincnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, y_train)).shuffle(x_train_cnn.shape[0],reshuffle_each_iteration=True).batch(128)
            
            if not data_load:   
                trainmlp, _, traincnn, _, y_train, _, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug = prd.prep_train_data(sess,raw,params)
                with open('train/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p', 'wb') as f:
                    pickle.dump([x_train_cnn, x_train_mlp, x_train_lda, x_train_aug, y_train, y_train_lda, sess.scaler], f)

            # Train neural networks
            models = [mlp, mlp_beta, cnn]
            for model in models:
                if isinstance(model,CNN):
                    ds = traincnn
                else:
                    ds = trainmlp
                
                train_mod = get_train()

                for epoch in range(sess.epochs):
                    # Reset the metrics at the start of the next epoch
                    train_loss.reset_states()
                    train_accuracy.reset_states()

                    for x, y in ds:
                        train_mod(x, y, model, optimizer, train_loss, train_accuracy)

                    if epoch == 0 or epoch == sess.epochs-1:
                        print(
                            f'Epoch {epoch + 1}, '
                            f'Loss: {train_loss.result():.2f}, '
                            f'Accuracy: {train_accuracy.result() * 100:.2f} '
                        )
                del train_mod
        
            # Train aligned LDA
            y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

            mlp_enc = mlp.get_layer(name='enc')
            w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

            mlpbeta_enc = mlp_beta.get_layer(name='enc')
            w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aug)
            
            cnn_enc = cnn.get_layer(name='enc')
            temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
            temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
            w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aug)

            # Train LDA
            w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
            w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

            mlp_w = mlp.get_weights()
            mlpb_w = mlp_beta.get_weights()
            cnn_w = cnn.get_weights()

            with open('models/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p','wb') as f:
                pickle.dump([mlp_w, mlpb_w, cnn_w, w_mlp, c_mlp, w_mlpbeta, c_mlpbeta, w_cnn, c_cnn, w, c, w_aug, c_aug, sess.emg_scale, sess.scaler],f)
        else:
            with open('models/' + str(sess.sub_type) + str(sub) + '_' + str(sess.feat_type) + '.p','rb') as f:
                mlp_w, mlpb_w, cnn_w, w_mlp, c_mlp, w_mlpbeta, c_mlpbeta, w_cnn, c_cnn, w, c, w_aug, c_aug, sess.emg_scale, sess.scaler = pickle.load(f)

            mlp_enc = mlp.get_layer(name='enc')
            mlpbeta_enc = mlp_beta.get_layer(name='enc')
            cnn_enc = cnn.get_layer(name='enc')

        if test:
            # Load test data
            if data_load:
                try:
                    with open('test/' + str(sess.sub_type) + str(sess.sub) + '_' + str(sess.feat_type) + '_' + str(sess.test) + '.p', 'rb') as f:
                        x_test_lda, y_test, clean_size = pickle.load(f)
                except:
                    print('no testing data to load')
                    data_load = False

                x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],int(x_test_lda.shape[1]/6),-1)),(0,2,1))[...,np.newaxis]
                x_test_cnn = sess.scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)
                x_test_cnn = x_test_cnn.astype('float32')

                # Reshape for nonconvolutional SAE
                x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)
            if not data_load:
                x_test_cnn, x_test_mlp, x_test_lda, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)

                with open('test/' + str(sess.sub_type) + str(sess.sub) + '_' + str(sess.feat_type) + '_' + str(sess.test) + '.p', 'wb') as f:
                    pickle.dump([x_test_lda, y_test, clean_size],f)

            # workaround
            mlp(x_test_mlp[:2,...])
            mlp_beta(x_test_mlp[:2,...])
            cnn(x_test_cnn[:2,...])

            mlp.set_weights(mlp_w)
            mlp_beta.set_weights(mlpb_w)
            cnn.set_weights(cnn_w)
        
            # Test
            mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
            mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
            cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
            y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

            # LDA
            clean_lda, noisy_lda = eval_lda(w, c, x_test_lda, y_test_aligned, clean_size)
            # AUG-LDA
            clean_aug, noisy_aug = eval_lda(w_aug, c_aug, x_test_lda, y_test_aligned, clean_size)
            # MLP
            clean_mlp, noisy_mlp = eval_nn(x_test_mlp, y_test,mlp,clean_size)
            clean_mlplda, noisy_mlplda = eval_lda(w_mlp, c_mlp, mlp_test_aligned, y_test_aligned, clean_size)
            # MLP Beta
            clean_mlpb, noisy_mlpb = eval_nn(x_test_mlp, y_test,mlp_beta,clean_size)
            clean_mlpblda, noisy_mlpblda = eval_lda(w_mlpbeta, c_mlpbeta, mlpbeta_test_aligned, y_test_aligned, clean_size)
            # CNN
            clean_cnn, noisy_cnn = eval_nn(x_test_cnn, y_test,cnn,clean_size)
            clean_cnnlda, noisy_cnnlda = eval_lda(w_cnn, c_cnn, cnn_test_aligned, y_test_aligned, clean_size)

            print( 
                f'LDA ---- '
                f'Clean: {clean_lda * 100:.2f}, '
                f'Noisy: {noisy_lda * 100:.2f}'
                f'\nAUG ---- '
                f'Clean: {clean_aug * 100:.2f}, '
                f'Noisy: {noisy_aug * 100:.2f}'
                f'\nMLP ---- '
                f'Clean: {clean_mlp * 100:.2f}, '
                f'Noisy: {noisy_mlp * 100:.2f}, '
                f'LDA Clean: {clean_mlplda * 100:.2f}, '
                f'LDA Noisy: {noisy_mlplda * 100:.2f}'
                f'\nMLPB ---- '
                f'Clean: {clean_mlpb * 100:.2f}, '
                f'Noisy: {noisy_mlpb * 100:.2f}, '
                f'LDA Clean: {clean_mlpblda * 100:.2f}, '
                f'LDA Noisy: {noisy_mlpblda * 100:.2f}'
                f'\nCNN ---- '
                f'Clean: {clean_cnn * 100:.2f}, '
                f'Noisy: {noisy_cnn * 100:.2f}, '
                f'LDA Clean: {clean_cnnlda * 100:.2f}, '
                f'LDA Noisy: {noisy_cnnlda * 100:.2f}'
            )

            clean_all[sub-1,:] = np.stack((clean_lda,clean_aug,clean_mlp,clean_mlplda,clean_mlpb,clean_mlpblda,clean_cnn,clean_cnnlda))
            noisy_all[sub-1,:] = np.stack((noisy_lda,noisy_aug,noisy_mlp,noisy_mlplda,noisy_mlpb,noisy_mlpblda,noisy_cnn,noisy_cnnlda))
    else:
        print('no training or testing data')

In [35]:
clean_mav = cp.deepcopy(clean_all)
noisy_mav = cp.deepcopy(noisy_all)

In [37]:
clean_mav[:,1] = clean_all[:,0]

In [103]:
def dense(x_in, w, fxn = 'relu'):
    out = (x_in @ w[0]) + w[1]
    if fxn == 'relu':
        out *= (out > 0)
    elif fxn == 'softmax':
        ax = np.argmax(out.shape)
        out = np.exp(out) / np.sum(np.exp(out), axis=ax)
    return out

def bn(x_in, w):
    out = ((w[0] * (x_in - w[2])) / np.sqrt(w[3] + 0.001)) + w[1]
    return out

In [96]:
test2 = mlp_enc.bn1(mlp_enc.dense1(x_test_mlp[[0],:]))

In [108]:
arch = ['relu','bn','relu','bn','relu','bn','den','bn','softmax']

In [109]:
w = mlp.get_weights()
x = x_test_mlp[[0],:]
i = 0 
for l in arch:
    if l == 'bn':
        w_layer = w[i:i+4]
        x = bn(x, w_layer)
        i += 4
    else:
        w_layer = w[i:i+2]
        x = dense(x, w_layer, fxn = l)
        i += 2

In [107]:
test = mlp(x_test_mlp[[0],:])

In [95]:
den_w = mlp_enc.dense1.get_weights()
bn_w = mlp_enc.bn1.get_weights()
test = bn(dense(x_test_mlp[[0],:], den_w), bn_w)

In [111]:
np.argmax(test) - np.argmax(x)

0